# 本番用

In [1]:

try:
    if first: pass
except:
    first = True
    %cd ..


c:\public_folder\technopro


In [2]:
import os

INPUT = os.path.join('input', 'input')

import torch
torch.manual_seed(69)


In [44]:
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from time import time, sleep
import numpy as np

In [46]:



class AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.MaxPool2d(2, 2),

            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.MaxPool2d(2, 2),
            
            nn.Conv2d(256, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.MaxPool2d(2, 2),

            nn.Conv2d(512, 1024, 3, padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(),
        )

        self.decoder = nn.Sequential(
            nn.Conv2d(1024, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.Upsample(32),

            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.Conv2d(512, 512, 3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.Conv2d(512, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Upsample(64),

            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),

            nn.Conv2d(256, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Upsample(128),

            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),

            nn.Conv2d(128, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Upsample(256),

            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Conv2d(64, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),

            nn.Conv2d(64, 1, 3, padding=1),
            nn.BatchNorm2d(1),
            #nn.ReLU(),
            nn.Sigmoid(),

        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

import torch
from torchinfo import summary

model = AutoEncoder()

model = torch.load(os.path.join('output', f'big_damage.pth'))
model = model.to('cuda')


# MAIN

In [ ]:
from torchvision import datasets
from torch.utils.data import DataLoader

import os
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from PIL import Image


transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])


dataset = datasets.ImageFolder(os.path.join(INPUT, 'test'), transform=transform)
loader = DataLoader(dataset, batch_size=1, shuffle=False)


device = 'cuda'

fp = open('ans.csv', 'w') 

for i, (inputs,  labels) in enumerate(loader):
    inputs, labels = inputs.to(device), labels.to(device)
    img = inputs.detach().cpu().numpy()
    black = np.sum(img<(10/255))
    img = cv2.imread(dataset.imgs[i][0], 0)
    
    name = dataset.imgs[i][0].split('\\')[-1].split('.')[0]
    fp.write(name+',')

    if black <= 20:
    
        gray_x = cv2.Sobel(img, cv2.CV_32F, 1, 0, ksize=3)
        gray_y = cv2.Sobel(img, cv2.CV_32F, 0, 1, ksize=3)
        dst = np.sqrt(gray_x ** 2 + gray_y ** 2)

        a = np.copy(dst)
        a /= np.max(a)

        a = (a > 0.4) * 255
        a = a.astype(np.uint8)

        contours, _ = cv2.findContours(a, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        a = 0    
        f = False
        for c in contours:
            area = cv2.contourArea(c)
            if area >= 55: 
                a = max(a, area)
                f = True

    else:

        img = Image.open(dataset.imgs[i][0])
        
        # convert gray
        img = img.convert('L')
        
        img = transform(img)
        img = img.reshape((1,)+img.shape)
        img = img.to('cuda')

        out = model(img).detach().cpu().numpy()[0, 0]
        out = out < (10/255)

        mask = img[0, 0] < (10/255)
        mask = mask.detach().cpu().numpy()

        per = np.sum(mask * out) / (256*256)

        if per > 0.005: f = True
        else: f = False



    if f: fp.write('1\n')
    else: fp.write('0\n')


fp.close()




torch.Size([1, 1, 256, 256])